In [1]:
#Connect google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
 #Install some of the needed stuff
 !pip install transformers datasets scikit-learn torch accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
#Import other stuff
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score

from datasets import Dataset
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer
)
import torch

In [4]:
#Replace this with the topic classifirer path
save_path_topics = #Your path here

In [5]:
#Then load it!
topic_model = RobertaForSequenceClassification.from_pretrained(save_path_topics)
topic_tokenizer = RobertaTokenizer.from_pretrained(save_path_topics)

In [7]:
#Same for the pros vs cons model
save_path_sentiment = #Your path here

In [8]:
#Then I load this one too
sent_model = RobertaForSequenceClassification.from_pretrained(save_path_sentiment)
sent_tokenizer = RobertaTokenizer.from_pretrained(save_path_sentiment)

A few more steps to make sure that we can use the same labels

In [9]:
import joblib

In [10]:
label_encoder_topics = joblib.load(f"{save_path_topics}/label_encoder.joblib")

In [11]:
label_encoder_sentiment = joblib.load(f"{save_path_sentiment}/label_encoder.joblib")

Getting ready to classify input; first off, I write functons to use both models

In [12]:
def predict_topic(text):
    topic_model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    topic_model.to(device)

    inputs = topic_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)

    le = label_encoder_topics

    with torch.no_grad():
        outputs = topic_model(**inputs)

    pred = outputs.logits.argmax(dim=1).item()
    return le.inverse_transform([pred])[0]

In [13]:
def predict_type(text):
    sent_model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    sent_model.to(device)

    inputs = sent_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)

    le = label_encoder_sentiment

    with torch.no_grad():
        outputs = sent_model(**inputs)

    pred = outputs.logits.argmax(dim=1).item()
    return le.inverse_transform([pred])[0]

I add a function to split sentences, assuming that people we are going to analyze longer input as well. I use anthropic, but feel free to replace it with the LLM of your choice


In [14]:
import re

In [15]:
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 5.4 MB/s eta 0:00:00


In [16]:
from anthropic import Anthropic

In [17]:
# Initialize Anthropic client
client = Anthropic(api_key= #your key here )

In [18]:
#Very important to handle the output of the LLM when it splits sentences (e.g. in case you have to create a new dataset with one row per sentence)
def clean_and_parse_response(response_text):
    """Clean the LLM response in a safe way, making sure that after this, the output is usable to separate sentences."""
    try:
        # Remove possible backtick and pyhthone code indicators
        cleaned = re.sub(r'```python\s*|\s*```', '', response_text.strip())

        # Remove inverted commas
        cleaned = re.sub(r'^["\']|["\']$', '', cleaned)

        # Try to parse using ast.literal_eval
        try:
            return ast.literal_eval(cleaned)
        except:
            # If it fails, tries to split the text using common patterns
            if re.search(r'^\[.*\]$', cleaned):
                # Remove square brackets
                items = cleaned[1:-1]
                # Split commas, but not the ones inside the sentences
                parts = re.findall(r'\'[^\']*\'|"[^"]*"', items)
                # Cleans individual pieces of text
                return [p.strip('\'"') for p in parts if p.strip()]

            print(f"Failed to parse response: {cleaned}")
            return []

    except Exception as e:
        print(f"Parsing error: {e}")
        print(f"Problematic response: {response_text}")
        return []

In [20]:
#I use sonnet as its performance are very good from a quality perspective, but feel free to use Haiku if you handle larger dataset and want to save costs or simply if you want something faster
def split_reviews_sonnet (text, client):
    try:
        response = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=2000,
            temperature=0.5,
            messages=[
                {
                    "role": "user",
                    "content": f"""You are an expert in analyzing employee reviews.
                    Your task is to split the following review into meaningful individual sentences, to allow an aspect-based sentiment analysis.
                    Separate points related to different topics; separate also distinct opinions on the same topic; separate also distinct points related to distinct sentiments on different aspetcts of the same aspect of a company.
                    When you split a sentence in two, please repeat the subject in the second sentence if needed. For example, if I tell you that the coworkers are nice but we do not get to work together on meaningful projects, repeat the subject in the second sentence if needed.
                    Preserve the original meaning of the text and do not add new information. If the input review is just one sentence, just return it as it is.
                    Do not use apostrophes or inverted commas, as they might mess up the way i divide the review into a list; please avoid contractions, as they will cause the same problem.
                    Input review: {text}
                    Output the points as a valid Python list of strings, like: ['Point 1.', 'Point 2.', 'Point 3.'].
                    Do not add extra characters, headings, or unrelated text."""
                }
            ]
        )

        # I use here the previous function to handle the response
        result = clean_and_parse_response(response.content[0].text)

        # Make sure that the response is a list
        if not isinstance(result, list):
            print(f"Unexpected response type: {type(result)}")
            return []

        # Filter empty strings and spaces
        filtered_result = [s.strip() for s in result if s.strip()]

        return filtered_result

    except Exception as e:
        print(f"Processing error: {e}")
        print(f"Original text: {text}")
        return []

In [21]:
def analyze_review(review):
    review_list = split_reviews_sonnet(review, client)
    pros = []
    cons = []

    for review in review_list:
        predicted_type = predict_type(review)
        predicted_topic = predict_topic(review)
        if predicted_type == "pro":
            pros.append(predicted_topic)
        elif predicted_type == "con":
            cons.append(predicted_topic)

    print("Pros:")
    for pro in pros:
      print(f"+ {pro}")

    print("Cons:")
    for con in cons:
      print(f"- {con}")

    return pros, cons

Example of use with a single sentence; feel free to adapt base on your use case (e.g. iterate over a dataset)

In [22]:
review = "I really value the flexible working—it’s made a big difference. My team’s supportive, but cross-team communication can be patchy and cause delays. There are good development opportunities, though they’re not always easy to find."
pros, cons = analyze_review(review)

Pros:
+ occupational wellbeing & work/life balance
+ group dynamics
+ career development and advancement opportunities
Cons:
- collaboration, team structure and work distribution
- career development and advancement opportunities


In [23]:
review_list = split_reviews_sonnet(review, client)
print(review_list)

['I really value the flexible working because it has made a big difference.', 'My team is supportive.', 'Cross-team communication can be patchy and cause delays.', 'There are good development opportunities.', 'Development opportunities are not always easy to find.']
